# How to convert Solvency 2 XBRL-instances to HTML and CSV-files

This tutorial shows you how to convert a XBRL-instance of a Solvency 2 report to a csv-file. This enables you to use the data from the XBRL-instance in your Python environment.

* install the Arelle package

* download the Solvency 2 taxonomy from the EIOPA website

* get your XBRL-instance(s) (we use here the Solvency 2 XBRL instance examples from the EIOPA website)

* read the XBRL-instance and convert it to a csv-file

## Install the Arelle package

Clone the package from https://github.com/Arelle/Arelle.git

And install the package

## Download the Solvency 2 taxonomy from the EIOPA website

The file we downloaded was:
https://dev.eiopa.europa.eu/Taxonomy/Full/2.4.0/S2/EIOPA_SolvencyII_XBRL_Taxonomy_2.4.0_with_external_hotfix.zip

## Get your XBRL-instance(s)

We got ours from:
https://eiopa.europa.eu/regulation-supervision/insurance/reporting-format/data-point-model-and-xbrl. 

The test instances are here:
https://dev.eiopa.europa.eu/Taxonomy/Full/2.4.0/S2/EIOPA_SolvencyII_XBRL_Instance_documents_2.4.0.zip


## Read the XBRL-instance and convert reports to HTML-pages

In [ ]:
from arelle import ViewFileFactTable, ViewFileFactList, ModelManager, FileSource
from arelle import Cntlr, ModelXbrl, ModelDocument, ViewFileFormulae
from arelle import ViewFileRenderedGrid, ModelFormulaObject, RenderingEvaluator
from ViewFileRenderedGrid import *

In [ ]:
DATA_PATH = 'H:\\20_local_data\\xbrl\\'
RESULTS_PATH = 'H:\\50_results\\xbrl\\'
LANGUAGE = "en-GB"

In [ ]:
modelmanager = ModelManager.initialize(Cntlr.Cntlr())
modelmanager.defaultLang = LANGUAGE
modelmanager.formulaOptions = ModelFormulaObject.FormulaOptions()
modelmanager.loadCustomTransforms()

In [ ]:
filename = DATA_PATH + 'qrs_240_instance.xbrl'
filesource = FileSource.FileSource(filename)

In [ ]:
xbrl_instance = ModelXbrl.load(modelmanager,filename)

## Convert XBRL-instance to HTML

Now we have the xbrl-instance read

In [ ]:
tables = xbrl_instance.relationshipSet("Table-rendering").linkRoleUris

In [ ]:
RenderingEvaluator.init(xbrl_instance)
for item in list(tables):
    ViewFileRenderedGrid.viewRenderedGrid(xbrl_instance,
                 RESULTS_PATH + item[-13:]+".html", 
                 viewTblELR=item,
                 lang = LANGUAGE, 
                 sourceView = None,
                 diffToFile = False)

# Convert HTML to CSV

WORK IN PROGRESS

In [ ]:
from bs4 import BeautifulSoup
import urllib
import csv

soup = BeautifulSoup(open(RESULTS_PATH + "S.05.01.02.01.html"), 'lxml')
table = soup.select_one("table")
# # python3 just use th.text

headers = [th.text for th in table.select("td")]

# with open("out.csv", "w") as f:
#     wr = csv.writer(f)
#     wr.writerow(headers)
#     wr.writerows([[td.text.encode("utf-8") for td in row.find_all("td")] for row in table.select("tr + tr")])

In [ ]:
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

tab = table.findAll("",{"class": ["xAxisHdr"]})
for item in tab:
    if 'rowspan' in item.attrs:
        print(str(item.attrs['rowspan']) + ", " + str(item.contents[0])+ ",")
    else:
        print(str(item.contents[0])+": ")

tab = table.findAll("",{"class": ["yAxisHdr", "yAxisHdrWithLeg", "cell"]})
for item in tab:
    if item.attrs['class'][0] == 'cell':
        if item.contents[0] != "\xa0":
            print(locale.atof(item.contents[0]))
        else:
            print("nan")
    else:
        if 'colspan' in item.attrs:
            print(str(item.attrs['colspan']) + ", " + str(item.contents[0])+ ",", end='')
        else:
            print(str(item.contents[0])+": ", end='')

## List validation rules in taxonomy

In [ ]:
import pandas as pd

In [ ]:
ViewFileFormulae.viewFormulae(xbrl_instance, RESULTS_PATH + "formulae.csv", "header", None)

In [ ]:
formulae = pd.read_csv(RESULTS_PATH + "formulae.csv")

In [ ]:
formulae[formulae['Expression'].str[0:2]=="BV"]['Expression']